In [1]:
import org.apache.spark.sql.SparkSession

In [3]:
val spark = SparkSession.builder()
                        .master("yarn")
                        .appName("aas09-montoCarlo-explore-run")
                        .config("spark.jars","file://home/kranthidr5051/gitRepos/aas/ch06-lsa/target/ch06-lsa-2.0.0-jar-with-dependencies.jar")
                        .getOrCreate()

spark = org.apache.spark.sql.SparkSession@49165f1f


lastException: Throwable = null


Spark application_1534772926501_7000: Some(http://ip-172-31-38-146.ec2.internal:4042)

In [1]:
spark

Waiting for a Spark session to start...

Spark application_1534772926501_6986: Some(http://ip-172-31-38-146.ec2.internal:4041)

In [ ]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

In [ ]:
import scala.collection.JavaConversions._

In [ ]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getProperty("user.home").split("/")(2)

In [ ]:
val path = hdfs_home+"/dataSets/nyc_taxidata2013/trip_data_1.csv"

In [ ]:
import java.io.File
import java.time.LocalDate
import java.time.format.DateTimeFormatter

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.mllib.stat.KernelDensity
import org.apache.spark.sql.{Dataset, SparkSession}
import org.apache.spark.sql.functions
import org.apache.spark.util.StatCounter
import breeze.plot._
import org.apache.commons.math3.distribution.ChiSquaredDistribution
import org.apache.commons.math3.distribution.MultivariateNormalDistribution
import org.apache.commons.math3.random.MersenneTwister
import org.apache.commons.math3.stat.correlation.Covariance
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression

In [ ]:
val spark = SparkSession.builder().getOrCreate()
    val runRisk = new RunRisk(spark)

    val (stocksReturns, factorsReturns) = runRisk.readStocksAndFactors()
    runRisk.plotDistribution(factorsReturns(2))
    runRisk.plotDistribution(factorsReturns(3))

    val numTrials = 10000000
    val parallelism = 1000
    val baseSeed = 1001L
    val trials = runRisk.computeTrialReturns(stocksReturns, factorsReturns, baseSeed, numTrials,
      parallelism)
    trials.cache()
    val valueAtRisk = runRisk.fivePercentVaR(trials)
    val conditionalValueAtRisk = runRisk.fivePercentCVaR(trials)
    println("VaR 5%: " + valueAtRisk)
    println("CVaR 5%: " + conditionalValueAtRisk)
    val varConfidenceInterval = runRisk.bootstrappedConfidenceInterval(trials,
      runRisk.fivePercentVaR, 100, .05)
    val cvarConfidenceInterval = runRisk.bootstrappedConfidenceInterval(trials,
      runRisk.fivePercentCVaR, 100, .05)
    println("VaR confidence interval: " + varConfidenceInterval)
    println("CVaR confidence interval: " + cvarConfidenceInterval)
    println("Kupiec test p-value: " + runRisk.kupiecTestPValue(stocksReturns, valueAtRisk, 0.05))
    runRisk.plotDistribution(trials)